In [1]:
# from allennlp.data.token_indexers import PretrainedTransformerMismatchedIndexer
# import jsonpickle
# a= PretrainedTransformerMismatchedIndexer("distilbert-base-uncased")
# a_pickled = jsonpickle.dumps(a)
# b = jsonpickle.loads(a_pickled)
# b._tokenizer.get_vocab()

In [35]:
a=['column:foreign:management:department_id', 'column:foreign:management:head_id', 'column:number:department:budget_in_billions', 'column:number:department:num_employees', 'column:number:department:ranking', 'column:number:head:age', 'column:primary:department:department_id', 'column:primary:head:head_id', 'column:text:department:creation', 'column:text:department:name', 'column:text:head:born_state', 'column:text:head:name', 'column:text:management:temporary_acting', 'table:department', 'table:head', 'table:management']
a

['column:foreign:management:department_id',
 'column:foreign:management:head_id',
 'column:number:department:budget_in_billions',
 'column:number:department:num_employees',
 'column:number:department:ranking',
 'column:number:head:age',
 'column:primary:department:department_id',
 'column:primary:head:head_id',
 'column:text:department:creation',
 'column:text:department:name',
 'column:text:head:born_state',
 'column:text:head:name',
 'column:text:management:temporary_acting',
 'table:department',
 'table:head',
 'table:management']

In [36]:
a

['column:foreign:management:department_id',
 'column:foreign:management:head_id',
 'column:number:department:budget_in_billions',
 'column:number:department:num_employees',
 'column:number:department:ranking',
 'column:number:head:age',
 'column:primary:department:department_id',
 'column:primary:head:head_id',
 'column:text:department:creation',
 'column:text:department:name',
 'column:text:head:born_state',
 'column:text:head:name',
 'column:text:management:temporary_acting',
 'table:department',
 'table:head',
 'table:management']

In [33]:
def cmp_to_key(mycmp):
    'Convert a cmp= function into a key= function'
    class K:
        def __init__(self, obj, *args):
            self.obj = obj
        def __lt__(self, other):
            return mycmp(self.obj, other.obj) < 0
        def __gt__(self, other):
            return mycmp(self.obj, other.obj) > 0
        def __eq__(self, other):
            return mycmp(self.obj, other.obj) == 0
        def __le__(self, other):
            return mycmp(self.obj, other.obj) <= 0
        def __ge__(self, other):
            return mycmp(self.obj, other.obj) >= 0
        def __ne__(self, other):
            return mycmp(self.obj, other.obj) != 0
    return K

def cmp_f(x,y):
    _x = x.split(":")
    _y = y.split(":")
    if len(_x)==2:
        _x = _x[1]
    else:
        _x = _x[2]+":"+_x[3]
    if len(_y)==2:
        _y = _y[1]
    else:
        _y = _y[2]+":"+_y[3]
#     c = 
#     print(c)
    return (_x > _y) - (_x < _y)
#     return int(_x < _y)
    

In [42]:
b=sorted(a,key=cmp_to_key(cmp_f))
b

['table:department',
 'column:number:department:budget_in_billions',
 'column:text:department:creation',
 'column:primary:department:department_id',
 'column:text:department:name',
 'column:number:department:num_employees',
 'column:number:department:ranking',
 'table:head',
 'column:number:head:age',
 'column:text:head:born_state',
 'column:primary:head:head_id',
 'column:text:head:name',
 'table:management',
 'column:foreign:management:department_id',
 'column:foreign:management:head_id',
 'column:text:management:temporary_acting']

In [65]:
def add_peren(b):
    c = []
    d = [":".join(x.split(":")[1:]) for x in b]
    append_list = []
    for i,x in enumerate(d):
        curr = x
        if ":" in curr: #col
            col_type,_,col_name = curr.split(":")
            curr = f"{col_name}:{col_type}"
            c.append(curr)
            if (i+1)<len(d) and ":" in d[i+1]:
                c.append(",")
            else:
                c.append(")\n")
            append_list.extend([True,False])

        else:
    #         c.append(curr)

            c.append(curr)
            c.append("(")
            append_list.extend([True,False])
    return c,append_list
c,h = add_peren(b)
print("".join(c))
print(c)
h


department(budget_in_billions:number,creation:text,department_id:primary,name:text,num_employees:number,ranking:number)
head(age:number,born_state:text,head_id:primary,name:text)
management(department_id:foreign,head_id:foreign,temporary_acting:text)

['department', '(', 'budget_in_billions:number', ',', 'creation:text', ',', 'department_id:primary', ',', 'name:text', ',', 'num_employees:number', ',', 'ranking:number', ')\n', 'head', '(', 'age:number', ',', 'born_state:text', ',', 'head_id:primary', ',', 'name:text', ')\n', 'management', '(', 'department_id:foreign', ',', 'head_id:foreign', ',', 'temporary_acting:text', ')\n']


[True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False]

NameError: name 'cmp' is not defined

In [24]:
"ab" -"c"

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [9]:
#     print(x,y)

In [2]:
from tqdm import tqdm

In [3]:
# for i in tqdm(range(10000000000000)):
#     pass

In [4]:
import os
# os.environ[“CUDA_DEVICE_ORDER”]=“PCI_BUS_ID”
os.environ["CUDA_VISIBLE_DEVICES"]="1"
# experiment_name = "3_heads_lr3_keep_op_identity+agenda_enriched_all+lr_e3+mult_scalar_per_action+glove"
# experiment_name = "crappy-red-dhole"
# train_dataset = reader.read("dataset/train_spider.json")
from models.semantic_parsing.ratsql_encoder import RatsqlEncoder
# from models.semantic_parsing.gnn_encoder import GnnEncoder
from models.semantic_parsing.spider_decoder import SpiderParser
from allennlp.modules.seq2vec_encoders.boe_encoder import BagOfEmbeddingsEncoder

from allennlp.modules.attention import DotProductAttention
from allennlp.nn.beam_search import BeamSearch
from allennlp.modules.seq2seq_encoders.pass_through_encoder import PassThroughEncoder


from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper

import torch.optim as optim
from allennlp.training.trainer import Trainer
import torch
from allennlp.models.archival import Archive
import torch
from allennlp.common import Params
from allennlp.models.model import Model
from allennlp.common.params import with_fallback
from dataset_readers.spider_ratsql import SpiderRatsqlDatasetReader
from allennlp.data.vocabulary import Vocabulary


# from models.semantic_parsing.spider_parser import SpiderParser
# reader = SpiderRatsqlDatasetReader(tables_file="dataset/tables.json",max_instances=None)
reader = SpiderRatsqlDatasetReader(tables_file="dataset/tables.json",max_instances=1000)
# settings = Params.from_file(f"experiments/{experiment_name}/config.json")
# model = Model.load(config=settings, serialization_dir=f"experiments/{experiment_name}")


train_dataset = reader.read("dataset/train_spider.json")
vocab = Vocabulary.from_instances(train_dataset)

In [5]:
# with torch.cuda.device(0):
EMBEDDING_DIM = 768
HIDDEN_DIM = 768
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})


beam = BeamSearch(end_index=0,beam_size=10)

schema_encoder = RatsqlEncoder(encoder=PassThroughEncoder(768),entity_encoder=BagOfEmbeddingsEncoder(768),question_embedder=word_embeddings,action_embedding_dim=768)
# schema_encoder = GnnEncoder(encoder=PassThroughEncoder(200),entity_encoder=BagOfEmbeddingsEncoder(200),question_embedder=word_embeddings,action_embedding_dim=200)
model = SpiderParser(vocab=vocab,schema_encoder=schema_encoder, 
                     decoder_beam_search=beam,input_attention=DotProductAttention(),past_attention=DotProductAttention(),max_decoding_steps=10)

model.cuda()
1

1

In [ ]:
a=None
for inst in train_dataset:
    a = inst
    res_list = model.forward_on_instances([inst])
    print(res_list)
#     break

In [ ]:
len(train_dataset)

In [ ]:
# iterator = BasicIterator(batch_size=15)
# optimizer = optim.Adam(model.parameters(), lr=0.001)
# iterator.index_with(vocab)
# trainer = Trainer(model=model,
#                   optimizer=optimizer,
#                   iterator=iterator,
#                   train_dataset=train_dataset,
#                   validation_dataset=train_dataset,
#                   patience=10,
#                   num_epochs=1)
# trainer.train()

In [ ]:
a.fields    
    

In [ ]:
import  allennlp.nn.util as util
import torch

def batched_span_select(target: torch.Tensor, spans: torch.LongTensor) -> torch.Tensor:
    """
    The given `spans` of size `(batch_size, num_spans, 2)` indexes into the sequence
    dimension (dimension 2) of the target, which has size `(batch_size, sequence_length,
    embedding_size)`.
    This function returns segmented spans in the target with respect to the provided span indices.
    It does not guarantee element order within each span.
    # Parameters
    target : `torch.Tensor`, required.
        A 3 dimensional tensor of shape (batch_size, sequence_length, embedding_size).
        This is the tensor to be indexed.
    indices : `torch.LongTensor`
        A 3 dimensional tensor of shape (batch_size, num_spans, 2) representing start and end
        indices (both inclusive) into the `sequence_length` dimension of the `target` tensor.
    # Returns
    span_embeddings : `torch.Tensor`
        A tensor with shape (batch_size, num_spans, max_batch_span_width, embedding_size]
        representing the embedded spans extracted from the batch flattened target tensor.
    span_mask: `torch.BoolTensor`
        A tensor with shape (batch_size, num_spans, max_batch_span_width) representing the mask on
        the returned span embeddings.
    """
    # both of shape (batch_size, num_spans, 1)
    span_starts, span_ends = spans.split(1, dim=-1)

    # shape (batch_size, num_spans, 1)
    # These span widths are off by 1, because the span ends are `inclusive`.
    span_widths = span_ends - span_starts

    # We need to know the maximum span width so we can
    # generate indices to extract the spans from the sequence tensor.
    # These indices will then get masked below, such that if the length
    # of a given span is smaller than the max, the rest of the values
    # are masked.
    max_batch_span_width = span_widths.max().item() + 1

    # Shape: (1, 1, max_batch_span_width)
    max_span_range_indices = util.get_range_vector(max_batch_span_width, util.get_device_of(target)).view(
        1, 1, -1
    )
#     print(max_batch_span_width)
#     print(max_span_range_indices)
    # Shape: (batch_size, num_spans, max_batch_span_width)
    # This is a broadcasted comparison - for each span we are considering,
    # we are creating a range vector of size max_span_width, but masking values
    # which are greater than the actual length of the span.
    #
    # We're using <= here (and for the mask below) because the span ends are
    # inclusive, so we want to include indices which are equal to span_widths rather
    # than using it as a non-inclusive upper bound.
    span_mask = max_span_range_indices <= span_widths
#     raw_span_indices = span_ends - max_span_range_indices
    raw_span_indices = span_starts + max_span_range_indices
#     print(raw_span_indices)
#     print(target.size())
    # We also don't want to include span indices which are less than zero,
    # which happens because some spans near the beginning of the sequence
    # have an end index < max_batch_span_width, so we add this to the mask here.
    span_mask = span_mask & (raw_span_indices < target.size(1))
#     print(span_mask)
#     span_indices = torch.nn.functional.relu(raw_span_indices.float()).long()
    span_indices = raw_span_indices * span_mask
#     print(span_indices)
    
    # Shape: (batch_size, num_spans, max_batch_span_width, embedding_dim)
    span_embeddings = util.batched_index_select(target, span_indices)

    return span_embeddings, span_mask


In [ ]:

# torch.tensor
a = util.get_range_vector(45,-1).reshape([3,15,1])
print(a)
q = 2
t = 3
b = torch.tensor([[0,q-1],[q,q+t-1]])
# b = torch.tensor([[[1,3]],[[2,3]],[[3,4]]])
print(b)

In [ ]:
batched_span_select(a,b)

In [ ]:
util.batched_span_select(a,b)

In [ ]:
# list()
import numpy as np
b = a.fields['enc']
token_ids = [0]+b.tokens
c = a.fields['lengths'].array

In [6]:
import json
schema_path = "dataset/tables.json"
with open(schema_path, "r") as f:
    dbs_json_blob = json.load(f)
    for i,db in enumerate(dbs_json_blob):
        print(db)
        print(db['column_names_original'])
#         diff = set(x.lower().re )
        print(db['column_names'])
        if i == 1:
            break

{'column_names': [[-1, '*'], [0, 'perpetrator id'], [0, 'people id'], [0, 'date'], [0, 'year'], [0, 'location'], [0, 'country'], [0, 'killed'], [0, 'injured'], [1, 'people id'], [1, 'name'], [1, 'height'], [1, 'weight'], [1, 'home town']], 'column_names_original': [[-1, '*'], [0, 'Perpetrator_ID'], [0, 'People_ID'], [0, 'Date'], [0, 'Year'], [0, 'Location'], [0, 'Country'], [0, 'Killed'], [0, 'Injured'], [1, 'People_ID'], [1, 'Name'], [1, 'Height'], [1, 'Weight'], [1, 'Home Town']], 'column_types': ['text', 'number', 'number', 'text', 'number', 'text', 'text', 'number', 'number', 'number', 'text', 'number', 'number', 'text'], 'db_id': 'perpetrator', 'foreign_keys': [[2, 9]], 'primary_keys': [1, 9], 'table_names': ['perpetrator', 'people'], 'table_names_original': ['perpetrator', 'people']}
[[-1, '*'], [0, 'Perpetrator_ID'], [0, 'People_ID'], [0, 'Date'], [0, 'Year'], [0, 'Location'], [0, 'Country'], [0, 'Killed'], [0, 'Injured'], [1, 'People_ID'], [1, 'Name'], [1, 'Height'], [1, 'Weigh

In [ ]:
# [0]+token_ids

In [ ]:
y = b.tokens[0]
# vars(y)
y.text_id

In [ ]:
# print(c.array)
offsets = get_offsets(c)
res = [token_ids[offsets[j][0]:offsets[j][1] + 1] for j in range(len(offsets))]
# print(res)
# print()
# for x in [[vocab.get_token_from_index(y,"tags") for y in x] for x in res]:
for x in res:
    print([y.text_id for y in x])
    print(x)

In [ ]:
torch.Size([1, 205, 2])
torch.Size([1, 205, 768])
tensor([[ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 16, 13, 12, 12, 12, 14, 14,
         13, 12, 13, 12, 13, 13, 13,  4,  4,  4]], dtype=torch.int32) torch.Size([1, 28])
tensor([2129])
tensor([2116])
tensor([4641])
tensor([1997])
tensor([1996])
tensor([7640])
tensor([2024])
tensor([3080])
tensor([2084])
tensor([5179])
tensor([1029])
tensor([ 1026,  2828,  1024,  3793,  1028,  1008,  1026,  2795,  1011, 19802,
         1028,  1026,  2151,  1011,  2795,  1028])
tensor([ 1026,  2828,  1024,  2193,  1028,  2533,  8909,  1026,  2795,  1011,
        19802,  1028,  2533])
tensor([ 1026,  2828,  1024,  3793,  1028,  2171,  1026,  2795,  1011, 19802,
         1028,  2533])
tensor([ 1026,  2828,  1024,  3793,  1028,  4325,  1026,  2795,  1011, 19802,
         1028,  2533])
tensor([ 1026,  2828,  1024,  2193,  1028,  5464,  1026,  2795,  1011, 19802,
         1028,  2533])
tensor([ 1026,  2828,  1024,  2193,  1028,  5166,  1999, 25501,  1026,  2795,
         1011, 19802,  1028,  2533])
tensor([ 1026,  2828,  1024,  2193,  1028, 16371,  1001,  1001,  1049,  5126,
         1026,  2795,  1011, 19802])
tensor([ 1028,  2533,  1026,  2828,  1024,  2193,  1028,  2132,  8909,  1026,
         2795,  1011, 19802])
tensor([ 1028,  2132,  1026,  2828,  1024,  3793,  1028,  2171,  1026,  2795,
         1011, 19802])
tensor([ 1028,  2132,  1026,  2828,  1024,  3793,  1028,  2141,  2110,  1026,
         2795,  1011, 19802])
tensor([ 1028,  2132,  1026,  2828,  1024,  2193,  1028,  2287,  1026,  2795,
         1011, 19802])
tensor([ 1028,  2132,  1026,  2828,  1024,  2193,  1028,  2533,  8909,  1026,
         2795,  1011, 19802])
tensor([ 1028,  2968,  1026,  2828,  1024,  2193,  1028,  2132,  8909,  1026,
         2795,  1011, 19802])
tensor([ 1028,  2968,  1026,  2828,  1024,  3793,  1028,  5741,  3772,  1026,
         2795,  1011, 19802])
tensor([1028, 2968, 1026, 2795])
tensor([1028, 2533, 1026, 2795])
tensor([1028, 2132, 1026, 2795])

In [ ]:
a=None
for inst in train_dataset:
    a = inst
    break

In [ ]:
vocab = Vocabulary.from_instances(train_dataset)
b=a.fields['enc']
b.index(vocab)
tok_dict = b._indexed_tokens['tokens']
print(len(tok_dict['token_ids']))

In [ ]:
import numpy as np
c=a.fields['lengths'].array
token_ids = tok_dict['token_ids']
old_offsets = tok_dict['offsets']
print()

In [ ]:
to_str = lambda x: [vocab.get_token_from_index(y,"tags") for y in x]
lengths = np.array(list(c))+1
# offsets = np.cumsum(lengths-1)
# offsets
# print(lengths)

# offsets = list(zip(offsets,offsets)
# offsets

In [ ]:
# a_ = 
# b_=offsets+lengths

In [ ]:

def get_offsets(lengths):
    e = np.cumsum(([0]+list(lengths))[:-1])
    return list(zip(e+1,e+np.array(lengths)))

offsets = get_offsets(c)

In [ ]:
offsets

In [ ]:
# _token_indexers
# to_str = lambda x: vocab.get_token_from_index(x,"tags")
# print(c.array)
res = [token_ids[offsets[j][0]:offsets[j][1] + 1] for j in range(len(offsets))]
# print(res)
# print()
for x in [[vocab.get_token_from_index(y,"tags") for y in x] for x in res]:
    print(x)

In [ ]:
vocab

In [ ]:
vocab.get_token_from_index(102,'tags')
# vocab

In [ ]:
d= b._token_indexers['tokens']
d

In [ ]:
d.indices_to_tokens([23],vocab)

In [ ]:
len()

In [ ]:
model.forward_on_instances([a])

In [ ]:
import random

i=random.randint(0,len(train_dataset))
a=None
for inst in train_dataset[i:i+1]:
    a = inst
    res_list = model.forward_on_instances([inst])
    

In [ ]:
a.fields['desc'].metadata

In [ ]:
b= res_list[0]['initial_state']
    

In [ ]:
c= b.get_valid_actions()

In [ ]:
# c.get_valid_actions()
b.possible_actions

In [ ]:
len(train_dataset)

In [ ]:
import random
i=random.randint(0,len(train_dataset))
a=None
for inst in train_dataset[i:i+1]:
#     print(inst)
    a=inst
# b=a.fields['relation']
# b.array
b=a.fields['world']
c=a.fields['desc']
d=a.fields['item'] 
e = a.fields["schema"]
# b
# print(a.fields['utterance'])
for entity in b.metadata.db_context.knowledge_graph.entities:
    print(entity)

In [ ]:
for entity in (c.metadata['columns']+c.metadata['tables']):
def normalize_schema_constant(entity):
#     print(entity)
    col = "_".join(entity)
    col =  col.split("_<table-sep>_")
    if len(col)==1:
        return "_".join(entity)
#         print()
    else:
        table = col[1]
        col = col[0].split(">_")[1]
        return f"{table}@{col}"
#         print()
        
    

In [ ]:
e.entity_texts

In [ ]:
print(a.fields['valid_actions'])

In [ ]:
d.metadata.schema.columns[3]

In [ ]:
"asdas_adsdas".split()

In [ ]:
# print()


In [ ]:

# print(b)


In [ ]:
import json
from dataset_readers.ratsql_spider_bert import *

# builder = RelationBuilder()


db_path = "dataset/database"
pre = SpiderEncoderV2Preproc()
schemas, eval_foreign_key_maps = pre.load_tables(["dataset/tables.json"])
# for path in paths:
for db_id, schema in tqdm(schemas.items(), desc="DB connections"):
    sqlite_path = Path(db_path) / db_id / f"{db_id}.sqlite"
    source: sqlite3.Connection
    with sqlite3.connect(sqlite_path) as source:
        dest = sqlite3.connect(':memory:')
        dest.row_factory = sqlite3.Row
        source.backup(dest)
    schema.connection = dest
    
examples=[]
raw_data = json.load(open("dataset/train_spider.json"))
for entry in raw_data:
    item = SpiderItem(
        text=entry['question_toks'],
        code=entry['sql'],
        schema=schemas[entry['db_id']],
        orig=entry,
        orig_schema=schemas[entry['db_id']].orig)
#     desc = pre.preprocess_item(item,"train")
    examples.append(item)

        # Backup in-memory copies of all the DBs and create the live connections


In [ ]:
# examples
# 
import pandas as pd
print 
# 
import tqdm
for item in tqdm.tqdm(examples):
    desc = pre.preprocess_item(item,"train")
    q = desc['question']
    q_len = len(q)
    t = [x[0] for x in  desc['tables']]
    t_len = len(t)
    c = ["_".join(x) for x in  desc['columns']]
    c_len = len(c)
    enc = q+c+t
    print(enc)
    relation = pre.compute_relations(desc,len(enc),q_len,c_len,range(c_len+1),range(t_len+1))
#     with np.printoptions(threshold=np.inf):
#     print(np.array2string())

    arr = pd.DataFrame(relation)
#     print()
    break
# arr


In [ ]:
for v in a.items():
    print(v)

In [ ]:
b=a.fields['utterance']

In [ ]:
b.sequence_length()
b.tokens

In [ ]:
c=a.fields['schema']
c.__dict__

In [ ]:
builder.compute_relations(dict(),)

In [ ]:
arr.to_csv("t.csv")